## STEP 3 -Scrape Booking.com

Data to scrap:
 - hotel name,
 - Url to its booking.com page,
 - Its coordinates: latitude and longitude
 - Score given by the website users
 - Text description of the hotel

In [1]:
import numpy as np
import pandas as pd
import requests
from requests import get
from bs4 import BeautifulSoup
import json

In [2]:
url_base="https://www.booking.com/searchresults.html?ss={}&order=bayesian_review_score"
REQUEST_HEADER = {
                 "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 "
                  "Safari/537.36"}

In [3]:
url_base

'https://www.booking.com/searchresults.html?ss={}&order=bayesian_review_score'

In [4]:
cityGeo = pd.read_csv("weather_for_top5_city.csv")
cityGeo.head()

,From,To,CityID,CityName,CityLat,CityLon,Pop,FeelLike,Temp,Clouds,WindSpeed
0,2022-01-29 12:00:00,2022-02-05 12:00:00,12,Besancon,47.238022,6.024362,0.59500,3.53875,5.90000,88.500,4.19250
1,2022-01-29 12:00:00,2022-02-05 12:00:00,9,Chateau du Haut Koenigsbourg,48.249490,7.344296,0.58625,-0.38750,2.34000,90.625,3.65125
2,2022-01-29 12:00:00,2022-02-05 12:00:00,11,Eguisheim,48.044797,7.307962,0.58625,4.34750,6.15125,88.250,3.74250
3,2022-01-29 12:00:00,2022-02-05 12:00:00,10,Colmar,48.077752,7.357964,0.58375,4.08000,6.04750,89.375,3.93000
4,2022-01-29 12:00:00,2022-02-05 12:00:00,8,Strasbourg,48.584614,7.750713,0.56375,2.73750,5.65000,85.500,5.54500


In [5]:
cols =['CityID','HotelName','HotelLat', 'HotelLon','HotelURL','HotelScore','HotelDescription']
resultHotels = pd.DataFrame(columns = cols)
for indx, row in cityGeo.iterrows():
    resultHotel={'CityID':row['CityID'],'CityName':row['CityName']}
    request = requests.get(url_base.format(row['CityName']),headers=REQUEST_HEADER)
    soup = BeautifulSoup(request.content, "lxml")
    hotels = soup.find_all("div", {"data-testid": "property-card"})
    for elem in hotels[:20]:
        resultHotel.update({'HotelName':elem.find("div", {"data-testid":"title"}).text})
        hotelUrl =elem.a['href']
        resultHotel.update({'HotelURL':hotelUrl})
        hotelScore = elem.find("div", {"data-testid":"review-score"})
        if hotelScore is not None:
            resultHotel.update({'HotelScore':elem.find("div", {"data-testid":"review-score"}).find("div").text})
        hotelDescriptionTag = elem.find("span", {"data-testid":"distance"})
        if hotelDescriptionTag is not None:
            resultHotel.update({'HotelDescription':elem.find("span", {"data-testid":"distance"}).find_next("div").text})
        details_hotel = BeautifulSoup(requests.get(hotelUrl, headers=REQUEST_HEADER).text,"lxml")
        resultHotel.update({'HotelLat':details_hotel.find("p",{'id':"showMap2"}).a['data-atlas-latlng'].split(',')[0]})
        resultHotel.update({'HotelLon':details_hotel.find("p",{'id':"showMap2"}).a['data-atlas-latlng'].split(',')[1]})
        resultHotels = resultHotels.append(resultHotel,ignore_index=True) 

In [6]:
result = pd.DataFrame.from_dict(resultHotels,orient='columns')
print(result.shape)
result.head()

(100, 9)


,CityID,HotelID,HotelName,HotelLat,HotelLon,HotelURL,HotelScore,HotelDescription,CityName
0,12,NaN,Hôtel Restaurant Spa Le Sauvage,47.23171767,6.02759153,https://www.booking.com/hotel/fr/ho-tel-le-sau...,9.0,"Hôtel Restaurant Spa Le Sauvage, formerly the ...",Besancon
1,12,NaN,L'adresse à Besançon,47.24736834,6.03148374,https://www.booking.com/hotel/fr/l-39-adresse-...,9.7,"L'adresse à Besançon is situated in Besançon, ...",Besancon
2,12,NaN,Chambre d'Hôtes La Villa Molina,47.25044475,6.02088064,https://www.booking.com/hotel/fr/la-villa-moli...,9.3,Set in a tree-lined garden with a shaded terra...,Besancon
3,12,NaN,Les Fontenottes,47.23927500,6.03571600,https://www.booking.com/hotel/fr/les-fontenott...,9.5,"Situated in Besançon, near Saint-Jean Cathedra...",Besancon
4,12,NaN,Hôtel de Paris,47.23856537,6.02532506,https://www.booking.com/hotel/fr/de-paris-besa...,8.7,"Located in the centre of Besançon, Hôtel de Pa...",Besancon


In [7]:
result.to_csv("top20hotels_weather_data.csv", index=False)